# CNN Fashion Image Classifier

In [ ]:
%matplotlib inline
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import string
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
from scipy.ndimage.filters import gaussian_filter
from scipy import ndimage

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.preprocessing.image import (
    random_rotation, random_shift, random_shear, random_zoom,
    random_channel_shift,  img_to_array, ImageDataGenerator)
from keras import backend as K
from keras.datasets import fashion_mnist

# nice progress bars for learning
import itertools
from sklearn.metrics import confusion_matrix

In [ ]:
random_seed = 2018
np.random.seed(random_seed)  
tf.random.set_seed(random_seed)

### Generate train and test ds

In [ ]:
#input
epois_size = 5
batch_size = 128
rand_size = 1000

input_layer_sq = 28


In [ ]:
def plot_images(imgs, labels=None, rows=1, figsize=(20,8), fontsize=14):
    figure = plt.figure(figsize=figsize)
    cols = max(1,len(imgs) // rows-1)
    labels_present = False
    # checking if labels is a numpy array
    if type(labels).__module__ == np.__name__:
        labels_present=labels.any()
    elif labels:
        labels_present=True
    for i in range(len(imgs)):
        subplot = figure.add_subplot(rows, cols+1, i+1)
        # axis off, but leave a bounding box
        plt.tick_params(
            axis='both',
            which='both',
            bottom='off',
            top='off',
            left='off',
            right='off',
            labelbottom='off',
            labelleft='off')
        # plot labels if present
        if labels_present:
            subplot.set_title(labels[i], fontsize=fontsize)
        plt.imshow(imgs[i][:,:,0], cmap='Greys')
        
    plt.show()

In [ ]:
#define getNormalized data
def getTrainAndTestData():
    #variable
    img_dimension = 255
    color_dim = 1

    #get train and test dat
    rand_val = np.random.randint(rand_size)
    
    (train_ds, train_labels), (test_ds, test_labels) = fashion_mnist.load_data()
    
    print (f'Mnist train data shape: {train_ds.shape}, label shape: {train_labels.shape}')
    print (f'Mnist test data shape: {test_ds.shape}, label shape: {test_labels.shape}')
    print ('\n')
    
    #normalize data
    print (f'Before noramlization, index: {rand_val}, \ntrain data value: {train_ds[rand_val][0:1]}, \nLabelvalue: {train_labels[rand_val]}')

    #convert data
    norm_train_ds = np.asarray(train_ds / img_dimension, dtype=float)
    norm_test_ds = np.asarray(test_ds / img_dimension, dtype=float)
    
    #convert label
    norm_train_labels = to_categorical(train_labels)
    norm_test_labels = to_categorical(test_labels)

    print ('\n')
    print (f'After noramlization, index: {rand_val}, \ntrain data value: {norm_train_labels[rand_val][0:1]}, \nLabelvalue: {norm_test_labels[rand_val]}')
    print ('\n')
    
    #reshape
    final_train_ds = norm_train_ds.reshape(-1,input_layer_sq, input_layer_sq, color_dim)
    final_test_ds = norm_test_ds.reshape(-1,input_layer_sq, input_layer_sq, color_dim)
    
    #copy directly
    final_train_labels = norm_train_labels
    final_test_labels = norm_test_labels
    
    print (f'Final train data shape: {final_train_ds.shape}')
    print (f'Final test data shape: {final_test_ds.shape}')
    print ('\n')
    
    return (final_train_ds, final_train_labels), (final_test_ds, final_test_labels)

#invoke fn 

(train_ds, train_labels), (test_ds, test_labels) = getTrainAndTestData();



In [ ]:
#define training model (CNN)
def getTrainModel():
    
    #variable
    max_pool_size = 3
    output_count = 10

    lv1_filter_size = 32
    lv2_filter_size = lv1_filter_size * 2

    bias_size = 1
    kernel_size = 3
    hidden_layer_size = 512
    output_layer_size = 10

    activation_relu = 'relu'
    activation_softmax = 'softmax'


    model_loss_fn = 'categorical_crossentropy'
    model_optimizer = Adam(lr=1e-3)
    model_metrics = ["accuracy"]

    model = Sequential()
    
    
    ##Convolution 
    #Convolution layer 1a
    model.add(
        Conv2D(
            filters = lv1_filter_size,
            kernel_size = (kernel_size, kernel_size), 
            activation=activation_relu,
            input_shape = (input_layer_sq, input_layer_sq, bias_size)
        )
    )
    
    model.add(BatchNormalization(axis=-1))
    
    #Convolution layer 1b
    model.add(
        Conv2D(
            filters = lv1_filter_size, 
            kernel_size = (kernel_size,kernel_size), 
            activation= activation_relu
        )
    )
    
    model.add(BatchNormalization(axis=-1))
    
    model.add(
        MaxPool2D(strides=(max_pool_size,max_pool_size))
    )
    
    #Convolution layer 2a
    model.add(
        Conv2D(
            filters = lv2_filter_size, 
            kernel_size = (kernel_size,kernel_size), 
            activation= activation_relu
        )
    )
    
    model.add(BatchNormalization(axis=-1))
    
    
    #Convolution layer 2b
    model.add(
        Conv2D(
            filters = lv2_filter_size, 
            kernel_size = (kernel_size,kernel_size), 
            activation= activation_relu
        )
    )
    
    model.add(BatchNormalization(axis=-1))

    model.add(
        MaxPool2D(strides=(max_pool_size,max_pool_size))
    )
    
    #Flatten input as 1-d array
    model.add(Flatten())
    
    #Fully connected layer
    #Fully connected layer 1 
    model.add(
        Dense(
            hidden_layer_size, activation=activation_relu
        )
    )
    
    #Fully connected layer level 2 
    model.add(Dropout(0.2))
    model.add(
        Dense(
            hidden_layer_size, activation=activation_relu
        )
    )
    
    

    #output level NN
    model.add(
        Dense(
            output_layer_size, activation=activation_softmax
        )
    )

    #set compiler, cost fn and metric for model
    model.compile(
        loss=model_loss_fn, 
        optimizer = model_optimizer, 
        metrics=model_metrics
    )
    
    return model

#invoke fn 
trainModel = getTrainModel()
print (trainModel.summary())

In [ ]:
#train model
trainResult = trainModel.fit(
                         train_ds,
                         train_labels,
                         batch_size=batch_size,
                         epochs=epois_size,
                         verbose=1,
                         validation_data=(test_ds, test_labels),
                         shuffle=True
                    )

print('\n')
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(
            trainResult.history['val_loss'][-1], 
            trainResult.history['val_accuracy'][-1]
        )
     )

In [ ]:
#evaluate training and test set

evaResults = trainModel.evaluate(train_ds, train_labels, batch_size=batch_size)
print("Evaluate \nloss: {0:.4f}, final accuracy: {1:.4f}".format(
            evaResults[0], 
            evaResults[1]
        )
     )


# Predict and check result

In [ ]:
#predict test ds
predictions_class = trainModel.predict_classes(test_ds)


In [ ]:
#check result by printing out
rand_idx = np.random.randint(rand_size)
    
pred_result = str(predictions_class[rand_idx])
actual_result = str(np.argmax(test_labels[rand_idx]))

if(pred_result == actual_result):
    result_str = 'CORRECT'
else:
    result_str = 'WRONG!!!!!'

print(f'Random item: {rand_idx} ')
print(f'Guess Result: {result_str}')
print(f'Predict value: {pred_result},  Actual Value: {actual_result}')

img_title = result_str + " , Predict: "+ pred_result +", Actual: "+ actual_result                                                                             
img = test_ds[rand_idx]
plot_images([img],labels=[img_title],figsize=(5,5))

# Export model for Python invoke

In [ ]:
#save model
trainModel.save('model/CNN_fashion_image_classifier_v1.h5')